# Counting Email in a Database

**Counting Organizations**

This application will read the mailbox data (mbox.txt) and count the number of email messages per organization (i.e. domain name of the email address) using a database with the following schema to maintain the counts.

```sql
CREATE TABLE Counts (org TEXT, count INTEGER)
```

When you have run the program on mbox.txt upload the resulting database file above for grading.
If you run the program multiple times in testing or with different files, make sure to empty out the data before each run.

You can use this code as a starting point for your application: http://www.py4e.com/code3/emaildb.py.

The data file for this application is the same as in previous assignments: http://www.py4e.com/code3/mbox.txt.

Because the sample code is using an UPDATE statement and committing the results to the database as each record is read in the loop, it might take as long as a few minutes to process all the data. The commit insists on completely writing all the data to disk every time it is called.

The program can be speeded up greatly by moving the commit operation outside of the loop. In any database program, there is a balance between the number of operations you execute between commits and the importance of not losing the results of operations that have not yet been committed.

In [1]:
import sqlite3
import pandas as pd

def count_emails_org(file_name, database_name):
    conn = sqlite3.connect(f'./data/{database_name}')
    cur = conn.cursor()

    cur.execute(
                '''
                DROP TABLE IF EXISTS Counts
                '''
                )

    cur.execute(
                '''
                CREATE TABLE Counts (organization TEXT, count INTEGER)
                '''
                )

    with open(f'./data/{file_name}') as file:
        text = file.readlines()

    for line in text:
        if not line.startswith('From: '): 
            continue
        pieces = line.split()
        email = pieces[1]
        split_email = email.split('@')
        organization = split_email[-1]
        cur.execute(
                    '''
                    SELECT count 
                    FROM Counts 
                    WHERE organization = ?
                    ''', (organization,)
                    )
        row = cur.fetchone()
        if row is None:
            cur.execute(
                        '''
                        INSERT INTO Counts (organization, count)
                        VALUES (?, 1)
                        ''', (organization,)
                        )
        else:
            cur.execute(
                        '''
                        UPDATE Counts SET count = count + 1 WHERE organization = ?
                        ''', (organization,)
                        )

    query = '''
            SELECT organization, count 
            FROM Counts 
            ORDER BY count DESC
            '''

    df = pd.read_sql_query(query, conn)

    conn.commit()
    cur.close()

    return df

count_emails_df = count_emails_org("mbox.txt", "emaildb.sqlite")
count_emails_df.head(10)

,organization,count
0,iupui.edu,536
1,umich.edu,491
2,indiana.edu,178
3,caret.cam.ac.uk,157
4,vt.edu,110
5,uct.ac.za,96
6,media.berkeley.edu,56
7,ufp.pt,28
8,gmail.com,25
9,et.gatech.edu,17
